In [46]:
from rapidfuzz.distance import Levenshtein
import pandas as pd
from unicodedata import normalize
import re
from tqdm import tqdm

In [44]:
Levenshtein.normalized_similarity('Zapatillas Nike', 'Zapatillas Nike', weights=(1,1,2))

1.0

In [103]:
df2 = pd.read_csv('items_titles_test.csv')

In [54]:
df2

,ITE_ITEM_TITLE
0,Tênis Olympikus Esporte Valente - Masculino Kids
1,Bicicleta Barra Forte Samy C/ 6 Marchas Cubo C...
2,Tênis Usthemp Slip-on Temático - Labrador 2
3,Tênis Casual Feminino Moleca Tecido Tie Dye
4,Tênis Star Baby Sapatinho Conforto + Brinde
...,...
9995,Chuteira Futsal Oxn Velox 3 Infantil
9996,Sapatenis Casual Masculino Estiloso 24horas Co...
9997,Tênis Feminino Infantil Molekinha Tie Dye
9998,Tênis Feminino Leve Barato Ganhe 1 Colchonete ...


# Normalização do texto

In [112]:
def new_norm_corpus(text):
    text = normalize('NFKD', text).encode('ASCII','ignore').decode('ASCII')
    #text = ' '.join(s for s in text.split() if not any(c.isdigit() for c in s))
    text = re.sub(r'[^\w\s]','',text)
    text = re.sub(r"<[^>]*>",'',text)
    text = text.lower()
    return text

In [104]:
df2['ITE_ITEM_TITLE'] = df2.apply(lambda x: new_norm_corpus(x['ITE_ITEM_TITLE']), axis=1)

In [106]:
df2

,ITE_ITEM_TITLE
0,tenis olympikus esporte valente masculino kids
1,bicicleta barra forte samy c 6 marchas cubo c ...
2,tenis usthemp slipon tematico labrador 2
3,tenis casual feminino moleca tecido tie dye
4,tenis star baby sapatinho conforto brinde
...,...
9995,chuteira futsal oxn velox 3 infantil
9996,sapatenis casual masculino estiloso 24horas co...
9997,tenis feminino infantil molekinha tie dye
9998,tenis feminino leve barato ganhe 1 colchonete ...


In [116]:
saida = list()

In [117]:
for idx,title in tqdm(df2.iterrows()):
    
    for idx2,tit in df2.iterrows():
        distancia = Levenshtein.normalized_similarity(title['ITE_ITEM_TITLE'], tit['ITE_ITEM_TITLE'], weights=(1,1,2))
        if distancia > 0.5:
            saida.append(
            {'ITE_ITEM_TITLE_1':title['ITE_ITEM_TITLE'],
            'ITE_ITEM_TITLE_2':tit['ITE_ITEM_TITLE'],
             'SCORE':distancia
            }
            )

10000it [1:16:47,  2.17it/s]


Conforme enumciado do desafio, é solicitado um notebook que seja capaz de determine o quão semelhante dois titulos do conjunto "item_titles_test.csv", foi usado o algoritimo Levenshtein muito utilizado para avaliar a distancia entre duas palavras através do numero de edções, para essa entrega foi selecionado itens com similaridade maior que 0.5, para como o conjunto possui 10 mil registros, caso coloquemos um a um, teriamos 100 milhões de retornos. Porem retirando a regra de distancia > 0.5, é possivel obter o valor completo de 100 milhões de registros.

In [118]:
saida_df = DataFrameataFrame(saida)

In [119]:
saida_df

,ITE_ITEM_TITLE_1,ITE_ITEM_TITLE_2,SCORE
0,tenis olympikus esporte valente masculino kids,tenis olympikus esporte valente masculino kids,1.000000
1,tenis olympikus esporte valente masculino kids,tenis ous emergente copper essencial,0.506024
2,tenis olympikus esporte valente masculino kids,tenis usthemp spice tematico raposinhas,0.505747
3,tenis olympikus esporte valente masculino kids,tenis para homens de couro macio leve masc adu...,0.538462
4,tenis olympikus esporte valente masculino kids,tenis usthemp short tematico bulldog frances,0.521739
...,...,...,...
8927357,tenis polo ralph lauren modelo cantor low bran...,tenis grife gucci rhyton boca masculino,0.510204
8927358,tenis polo ralph lauren modelo cantor low bran...,tenis infantil star feet cano medio masculino ...,0.530973
8927359,tenis polo ralph lauren modelo cantor low bran...,sapatenis polo plus original tenis slip on ci...,0.534483
8927360,tenis polo ralph lauren modelo cantor low bran...,tenis em couro sapatenis solado preto ogochi m...,0.607143


In [120]:
saida_df.to_csv('SAIDA_SIMILARIDADE.csv',sep='|')